In [1]:
import numpy as np
import tensorflow as tf 
from tensorflow import keras
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Activation , Dense , Flatten , BatchNormalization,Dropout , Conv2D , MaxPool2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix
import itertools
import os
import shutil
import random
import glob
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter(action = 'ignore', category = FutureWarning)
%matplotlib inline

In [ ]:
train_path = "C:/Users/Aasisjuli/Desktop/data for training/chest_xray/train"
valid_path ="C:/Users/Aasisjuli/Desktop/data for training/chest_xray/val"
test_path="C:/Users/Aasisjuli/Desktop/data for training/chest_xray/test"

In [ ]:
train_batches= ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input)\
                .flow_from_directory(directory = train_path ,
                                     target_size=(224,224),
                                    classes=['NORMAL','PNEUMONIA'],
                                    batch_size=10)
valid_batches= ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input)\
                .flow_from_directory(directory = valid_path ,
                                     target_size=(224,224),
                                    classes=['NORMAL','PNEUMONIA'],
                                    batch_size=10)
test_batches= ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input)\
                .flow_from_directory(directory = test_path ,
                                     target_size=(224,224),
                                    classes=['NORMAL','PNEUMONIA'],
                                    batch_size=10)

In [ ]:
imgs,labels=next(train_batches)

In [ ]:
def plotImages(images_arr):
    fig, axes = plt.subplots(1, 10, figsize=(20,20))
    axes = axes.flatten()
    for img, ax in zip( images_arr, axes):
        ax.imshow(img)
        ax.axis('off')
    plt.tight_layout()
    plt.show()

In [ ]:
plotImages(imgs)
print(labels)

In [ ]:
vgg16_model=tf.keras.applications.vgg16.VGG16()

In [ ]:
from tensorflow.keras.applications import DenseNet121

In [ ]:
from tensorflow.keras.applications.densenet import preprocess_input

In [ ]:
densenet = DenseNet121(input_shape=[224,244,3],weights='imagenet',include_top=False)

In [ ]:
densenet.summary()


In [ ]:

for layer in densenet.layers:
    layer.trainable=False

In [ ]:
x=Flatten()(densenet.output)
prediction = Dense(2,activation='softmax')(x)

In [ ]:

model = keras.Model(inputs=densenet.input, outputs=prediction)

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.0001),loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
model.fit(x=train_batches , validation_data=valid_batches , validation_steps=len(valid_batches), epochs=5 , verbose=2)

In [ ]:
predictions = model.predict(x=test_batches,verbose=0,steps=len(test_batches))

In [ ]:
test_batches.classes

In [ ]:
cm = confusion_matrix(y_true= test_batches.classes , y_pred=np.argmax(predictions,axis=-1))

In [ ]:
cm_plot_labels=['normal','pnuemonia']
plot_confusion_matrix(cm=cm, classes=cm_plot_labels,title='Confusion Matrix')


In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
            horizontalalignment="center",
            color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')